# EE4375 Using GMSH for Geometry Definition and Mesh Creation 

This notebook gives guidelines for using install and running [GMSH](https://gmsh.info) in the EE4375 course.

## Import Packages 

In [2]:
try
    using Gmsh: gmsh
catch
    using gmsh
end 

## Section 1: Installation 

Install the [GMSH.jl](https://github.com/JuliaFEM/Gmsh.jl/tree/master) package. 

The initialization and finalization of the GMSH engine requires calling `initialize()` and `finalize()`, respectively. 

## Section 2: Geometry Definition

### Using Geometry Primitives 

How to create geometries in 2D by the sequence of following steps
1. define a <b>point</b> by specifying three inputs. The first input is a tuple of (x,y,z)-coordinates of the point. The second is the mesh density in the vicinity of the point (here defined by lc). The third is a unique label for the point. For example: to add a point in th origin with mesh density lc and label 1, use `addPoint(0, 0, 0, lc, 1)`. Notes: labels serve as unique references in subsequent function call. Note that even when defining two-dimensional geometries, a point has a z-coordinate;   

1. define a <b>line</b> (or straight segment) by providing three inputs. The first is the label of the starting point. The second is the label of the ending point. The third is a unique label of the line. For example, to define a line starting in a point labeled by 1, ending in a point labeled by 2 and with a label 1 for this line, use `addLine(1, 2, 1)`. Note: a line has an orientation. The line starting in the point with label 2 and ending in the point with label 1 is the oppossite of the line defined before. Curved segment (e.g. to create circles) will be discussed later;  

1. define a <b>closed loop</b> by providing two inputs. The first is a ordered list on oriented lines. This list is defined by square brackets (by concatenation of lines or segments) and providing labels. The second argument is a unique label for the closed loop. For example: to add a closed loop formed by lines with labels 1, 2, 3 and 4 and with label 1 for this loop, use `addCurveLoop([1, 2, 3, 4], 1)`. Closed loops are typically used as boundary of a two-dimensional domain; 

1. define a <b>surface</b> by providing two inputs. The first is a list (hence square brackets) of curved loops at th boundary of the surface. The second is a label for this surface. For example: to add a surface bounded by closed loop 1 and with label 1, use `addPlaneSurface([1], 1)`;

1. synchronize the geo module of GMSH by calling the 'synchronize()' function;

### Exercise: Create Geometry of a Triangle 

We create the geometry of a triangle. The mesh generation is postponed to a later stage.  

In [3]:
#..1/5: initialize gmsh 
gmsh.initialize()

#..2/5: generate geometry 
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("square")
#..set mesh density parameter 
lc1 = .1
lc2 = .5
lc3 = .01
#..define four points via (x,y,z) coordinates 
p1 = gmsh.model.geo.addPoint(0, 0, 0, lc1, 1)
p2 = gmsh.model.geo.addPoint(2., 0,  0, lc2, 2)
p3 = gmsh.model.geo.addPoint(1., 1., 0, lc3, 3)
#..define four edges by connecting point labels pairwise  
l1 = gmsh.model.geo.addLine(1, 2, 1)
l2 = gmsh.model.geo.addLine(2, 3, 2)
l3 = gmsh.model.geo.addLine(3, 1, 3)
#..define curved loop by connecting four edge labels  
loop = gmsh.model.geo.addCurveLoop([1, 2, 3], 1)
#..define surface by curved loop 
surf = gmsh.model.geo.addPlaneSurface([1], 1)

#..3/5: synchronize the CAD model 
gmsh.model.geo.synchronize()

#..4/5: write mesh to mesh and visualize the mesh  
#..if true, write mesh to file for further processing 
if (false) gmsh.write("my-mesh-file.msh") end 
#..if true, visualize mesh through the GUI 
if (false) gmsh.fltk.run() end 

#..5/5: finalize gmsh 
gmsh.finalize()

-------------------------------------------------------
Version       : 4.10.2
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 19700101
Build host    : amdci7
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.6.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


### Exercises

Giving code that follows: 
1. transform triangle into quadrilateral;
1. create quadrilateral with curved edge;
1. create quadrilateral minus interior triangle;
1. create quadrilateral with seperate regions inside and outside of triangle; 

### Examples  

1. moving geometry (linear actuator - rotating machines) using scripting;
1. curved shaped using spline using scripting to enable shape optimization at a later stage; 
1. 3D geometries using SALOME or Open-Cascade for parametric CAD models;

## Section 3:  Mesh Generation 

1. theory: what algorithms are available to create triangular, quadrilaterals meshes and 3D meshes;
1. use gmsh.generate(1) to create boundary mesh. use gmsh.generate(2) to create boundary+interior mesh.  
1. use gmsh.generate to generate mesh with triangles, quadrilaterals, mixed, first or second order 
1. how to use these algorithms in practical cases; 
1. how to read the output of gmsh.generate(). What is an element for gmsh (1D lines segments plus 2D triangular elements)

###  Create Geometry of an Interval, Generate Mesh and Check Output Generated 

1. observe the non-uniformity in the mesh caused by unequal lc settings in left and right node; 
1. what does number of nodes and elements refer to? 
1. what happens in case interior points are added to the interval?
1. what happens in case higher order elements are generated? 

In [44]:
#..1/6: initialize gmsh 
gmsh.initialize()

#..2/6: generate geometry 
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("bar")
#..set mesh density parameter 
lc1 = .1
lc2 = .5
#..define four points via (x,y,z) coordinates 
p1 = gmsh.model.geo.addPoint(0, 0, 0, lc1, 1)
p2 = gmsh.model.geo.addPoint(2., 0,  0, lc2, 2)
#..define four edges by connecting point labels pairwise  
l1 = gmsh.model.geo.addLine(1, 2, 1)

#..3/6: synchronize the CAD model 
gmsh.model.geo.synchronize()

#..4/6: generate two-dimensional mesh 
gmsh.option.setNumber("Mesh.ElementOrder", 1)
gmsh.model.mesh.generate(1)

#..5/6: write mesh to mesh and visualize the mesh  
#..if true, write mesh to file for further processing 
if (true) gmsh.write("bar.msh") end 
#..if true, visualize mesh through the GUI 
if (false) gmsh.fltk.run() end 

#..6/6: finalize gmsh 
gmsh.finalize()

Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Done meshing 1D (Wall 0.00238475s, CPU 0.00225s)
Info    : 10 nodes 11 elements
Info    : Writing 'bar.msh'...
Info    : Done writing 'bar.msh'


###  Create Geometry of a Triangle, Generate Mesh and Check Output Generated 

1. what does number of elements refer to? 

In [9]:
#..1/6: initialize gmsh 
gmsh.initialize()

#..2/6: generate geometry 
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("square")
#..set mesh density parameter 
lc1 = .1
lc2 = .5
lc3 = .01
#..define four points via (x,y,z) coordinates 
p1 = gmsh.model.geo.addPoint(0, 0, 0, lc1, 1)
p2 = gmsh.model.geo.addPoint(2., 0,  0, lc2, 2)
p3 = gmsh.model.geo.addPoint(1., 1., 0, lc3, 3)
#..define four edges by connecting point labels pairwise  
l1 = gmsh.model.geo.addLine(1, 2, 1)
l2 = gmsh.model.geo.addLine(2, 3, 2)
l3 = gmsh.model.geo.addLine(3, 1, 3)
#..define curved loop by connecting four edge labels  
loop = gmsh.model.geo.addCurveLoop([1, 2, 3], 1)
#..define surface by curved loop 
surf = gmsh.model.geo.addPlaneSurface([1], 1)

#..3/6: synchronize the CAD model 
gmsh.model.geo.synchronize()

#..4/6: generate two-dimensional mesh 
gmsh.model.mesh.generate(1)

#..5/6: write mesh to mesh and visualize the mesh  
#..if true, write mesh to file for further processing 
if (false) gmsh.write("my-mesh-file.msh") end 
#..if true, visualize mesh through the GUI 
if (true) gmsh.fltk.run() end 

#..6/6: finalize gmsh 
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 40%] Meshing curve 2 (Line)
Info    : [ 70%] Meshing curve 3 (Line)
Info    : Done meshing 1D (Wall 0.00926367s, CPU 0.008652s)
Info    : 58 nodes 61 elements
-------------------------------------------------------
Version       : 4.10.2
License       : GNU General Public License
Build OS      : MacOSX-sdk
Build date    : 19700101
Build host    : amdci7
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.6.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
------------------------------

In [14]:
#..3/4: generate two-dimensional mesh 
#gmsh.option.setNumber("Mesh.ElementOrder", 2)
#gmsh.option.setNumber("Mesh.SecondOrderLinear", 1)

### Geometry Definition and Mesh Generation Using GMSH  
See [gijswl/ee4375_fem_ta/tree/main/general](https://github.com/gijswl/ee4375_fem_ta/tree/main/general); (Domenico needs to reread)

Existing Documentation 
1. [Post on stackexchange: Connectivity matrix in Finite Element Method in Triangular elements](https://scicomp.stackexchange.com/questions/30169/connectivity-matrix-in-finite-element-method-in-triangular-elements): small mesh on square geometry with clear illustration labeling nodes and elements;
1. [21 Tutorial Examples from GMSH tutorials/julia](https://gitlab.onelab.info/gmsh/gmsh/-/tree/master/tutorials/julia)
1. [2 Test Examples t1.geo and 16.go from GMSH.jl](https://github.com/JuliaFEM/Gmsh.jl)

### Add as exercises on the use of GMSH  
1. semi-circle from Victoria Hernandez (parametrize geometry with r and a, take r=0.133 m and a = 0.01 m) (need to solve radiation problem using gridap.jl); 
1. STEDIN distribution transformer (cfd. Max van Dijk);
1. linear actuator (need to recover dimensions from papers); 
1. machine from Jianning Dong (see hard drive); 
1. Contactless energy transfer - Litz wire 

## Section 4: Writing Mesh to File 

Using gmsh.write("mesh-file.msh"). 

## Section 5: Visualizing the Mesh 

1. using the GMSH GUI 
1. writing the mesn to file and load into Paraview 

## Section 6: Reading Mesh From File 

Using gmsh.open("mesh-file.msh").  

## Section 7: Adding Physical Groups   

In [ ]:
#..1/7: initialize gmsh 
gmsh.initialize()

#..2/7: generate geometry 
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.add("square")
#..set mesh density parameter 
lc = .1
#..define four points via (x,y,z) coordinates 
p1 = gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
p2 = gmsh.model.geo.addPoint(1., 0,  0, lc, 2)
p3 = gmsh.model.geo.addPoint(1., 1., 0, lc, 3)
p4 = gmsh.model.geo.addPoint(0, 1., 0, lc, 4)
#..define four edges by connecting point labels pairwise  
l1 = gmsh.model.geo.addLine(1, 2, 1)
l2 = gmsh.model.geo.addLine(2, 3, 2)
l3 = gmsh.model.geo.addLine(3, 4, 3)
l4 = gmsh.model.geo.addLine(4, 1, 4)
#..define curved loop by connecting four edge labels  
loop = gmsh.model.geo.addCurveLoop([1, 2, 3, 4], 1)
#..define surface by curved loop 
surf = gmsh.model.geo.addPlaneSurface([1], 1)

#..3/7: synchronize the CAD model 
gmsh.model.geo.synchronize()

#..4/7: assign physical groups
gmsh.model.addPhysicalGroup(1, [l1], -1, "bottom")
gmsh.model.addPhysicalGroup(1, [l2], -1, "right")
gmsh.model.addPhysicalGroup(1, [l3], -1, "top")
gmsh.model.addPhysicalGroup(1, [l4], -1, "left")
gmsh.model.addPhysicalGroup(2, [surf], -1, "omega")

#..5/7: generate two-dimensional mesh 
gmsh.model.mesh.generate(2)

#..6/7: write mesh to mesh and visualize the mesh  
#..if true, write mesh to file for further processing 
if (true) gmsh.write("data/square.msh") end 
#..if true, visualize mesh through the GUI 
if (false) gmsh.fltk.run() end 

#..7/7: finalize gmsh 
gmsh.finalize()

## Section 8: GMSH Mesh Traversals

1. how is connectivity of 2D/surface mesh encoded? How to traverse over elements in the 2D mesh?
1. element__connectivity is a vector of (unsigned long) integers as long as the number of elements times the number of elements on each mesh. For instance for 1D line elements, length of element_connectivity is 2* nelement. The global node IDs are stacked consecutively in element__connectivity.    
1. what happens in case of first and second order triangle / quadrilateral meshes?
1. can the element area and Jacobian be extracted?
1. how is connectivity of 1D/boundary mesh encoded? How to traverse over elements in the 1D mesh?

In [46]:
gmsh.finalize()

In [49]:
#..1/4: Finalize gmsh
gmsh.initialize()

#..2/4: Read mesh from file
gmsh.open("bar.msh")

#..3/4: perform loop over the elemements 
element_types, element_ids, element_connectivity = gmsh.model.mesh.getElements(1)
nelements = length(element_ids[1])
  
for element_id in 1:nelements

    #....retrieve global numbering of the local nodes of the current element
    node1_id = element_connectivity[1][2*(element_id-1)+1]
    node2_id = element_connectivity[1][2*(element_id-1)+2]

    if (true)
      println("on element ", element_id, " node-1 has global number ", node1_id)
      println("on element ", element_id, " node-2 has global number ", node2_id)
      println(" ")
    end 

end 

#..4/4: Finalize gmsh
gmsh.initialize()

Info    : Reading 'bar.msh'...
Info    : 3 entities
Info    : 10 nodes
Info    : 11 elements
Info    : Done reading 'bar.msh'
on element 1 node-1 has global number 1
on element 1 node-2 has global number 3
 
on element 2 node-1 has global number 3
on element 2 node-2 has global number 4
 
on element 3 node-1 has global number 4
on element 3 node-2 has global number 5
 
on element 4 node-1 has global number 5
on element 4 node-2 has global number 6
 
on element 5 node-1 has global number 6
on element 5 node-2 has global number 7
 
on element 6 node-1 has global number 7
on element 6 node-2 has global number 8
 
on element 7 node-1 has global number 8
on element 7 node-2 has global number 9
 
on element 8 node-1 has global number 9
on element 8 node-2 has global number 10
 
on element 9 node-1 has global number 10
on element 9 node-2 has global number 2
 


In [53]:
element_connectivity[1]

18-element Vector{UInt64}:
 0x0000000000000001
 0x0000000000000003
 0x0000000000000003
 0x0000000000000004
 0x0000000000000004
 0x0000000000000005
 0x0000000000000005
 0x0000000000000006
 0x0000000000000006
 0x0000000000000007
 0x0000000000000007
 0x0000000000000008
 0x0000000000000008
 0x0000000000000009
 0x0000000000000009
 0x000000000000000a
 0x000000000000000a
 0x0000000000000002

## Section 9: Auxiliary Tools 

1. [FerriteGmsh.jl](https://github.com/Ferrite-FEM/FerriteGmsh.jl)
1. [GridapGmsh.jl](https://github.com/gridap/GridapGmsh.jl)
1. [Meshes.jl](https://github.com/JuliaGeometry/Meshes.jl)
1. [ExtendableGrids.jl](https://github.com/j-fu/ExtendableGrids.jl)
1. [WriteVTK.jl](https://juliavtk.github.io/WriteVTK.jl/dev/): how to adapt to second order elements 
5. [MakieCore.mesh function](https://docs.makie.org/stable/reference/plots/mesh#colormap) for plotting 

## Section 10: Alternatives for GMSH 

1. [Triangle.jl](https://cvdlab.github.io/Triangle.jl/)
2. [TetGen.jl](https://juliageometry.github.io/TetGen.jl/stable/)
3. [DistMesh.jl](https://distmesh.juliageometry.org/dev/)

## References